In [1]:
from carbon.helpers.stdimports import *
from carbon.helpers.soltest import SolTestBase
from carbon.helpers.floatint import *
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SolTestBase))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonFloatInt32))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonOrderUI))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(P))
from math import log2, floor, ceil, sqrt
print_version(require="2.3.3")

[stdimports] imported np, pd, plt, os, sqrt, exp, log
SolTestBase v1.0 (11/Feb/2023)
CarbonFloatInt32 v1.0 (10/Feb/2023)
CarbonOrderUI v1.7.1 (12/Feb/2023)
CarbonPair v2.2 (11/Feb/2022)
Carbon v2.3.3-BETA1 (12/Feb/2022)


# Solidity Curve Testing (NBTest058)

we consider by source and by target **from the point of view of the TRADER**, therefore _by source_ fixes _dx_ and calculates _dy_ and vice versa.

**dx from dy ("by target")**

$$
\Delta x \,(\Delta y) = \frac{\Delta y z^2}{(Ay+Bz)(Ay+Bz-A\Delta y)} =_{A=0} \frac{\Delta y} {B^2}
$$



**dy from dx ("by source")**

$$
\Delta y \,(\Delta x) = \frac{\Delta x(Ay+Bz)^2}{A\Delta x(Ay+Bz)+z^2} =_{A=0} \frac{\Delta x(Bz)^2}{z^2} = \Delta x B^2
$$

[doc][doc]


[doc]:https://docs.google.com/document/d/1x4ZbbS3nIRSJ0ojaOrcBhOc-_eOTga6BajDjK45o7u4/edit

In [2]:
FI32 = CarbonFloatInt32
FI40 = CarbonFloatInt40

## Demo and test of yzABS in CarbonOrderUI and pair decimals

In [3]:
ETHUSDC = P("ETH/USDC").sd(18,6)
assert ETHUSDC.decimals == {'ETH': 18, 'USDC': 6, '_TKNB': 18, '_TKNQ': 6, '_DIFFQB': -12}
assert ETHUSDC.decdiffqb == -12
assert P(ETHUSDC).has_decimals
assert ETHUSDC.price_convention == 'USDC per ETH'
ETHUSDC

P('ETH/USDC').sd(18,6)

In [4]:
oui = CarbonOrderUI.from_prices(ETHUSDC, "ETH", 1500, 2000, 1, 0.5)
r = oui.yzABS(sx=10, verbose=True)
ddf, ddf2 = 10**12, 10**6
assert r.y == 0.5 * 10**18
assert r.z == 10**18
assert r.A == int(oui.S * ddf2 * r.S)
assert r.B == int(oui.B * ddf2 * r.S)
assert r.S == 2**10
assert oui.descr() == 'Sell ETH buy USDC from 1500.0000 to 2000.0000 USDC per ETH'

[yzABS] pair=P('ETH/USDC').sd(18,6), y=ETH(18), x=USDC(6)
[yzABS] scale = 2**10 = 1024
[yzABS] y=0.5 -> y_wei=5e+17 [ETH-wei]
[yzABS] yint=1 -> z_wei=1000000000000000000 [ETH-wei]
[yzABS] pa_raw=0.0006666666666666666 ETH per USDC -> 666666666 ETH-wei per USDC-wei
[yzABS] pb_raw=0.0005 ETH per USDC -> 500000000 ETH-wei per USDC-wei
[yzABS] a=0.0034592091997182155 -> 3459.2091997182156 * scale = 3542230
[yzABS] b=0.022360679774997897 -> 22360.679774997898 * scale = 22897336
[yzABS] yzABS = r(y=5e+17, z=1000000000000000000, A=3542230, B=22897336, S=1024)


In [5]:
oui = CarbonOrderUI.from_prices(ETHUSDC, "USDC", 1000, 500, 1100, 790)
r = oui.yzABS(sx=40, verbose=True)
ddf, ddf2 = 10**-12, 10**-6
assert r.y == 790 * 10**6
assert r.z == 1100 * 10**6
assert r.A == int(oui.S * ddf2 * r.S)
assert r.B == int(oui.B * ddf2 * r.S)
assert r.S == 2**40

[yzABS] pair=P('ETH/USDC').sd(18,6), y=USDC(6), x=ETH(18)
[yzABS] scale = 2**40 = 1099511627776
[yzABS] y=790 -> y_wei=790000000 [USDC-wei]
[yzABS] yint=1100 -> z_wei=1100000000 [USDC-wei]
[yzABS] pa_raw=1000.0 USDC per ETH -> 0 USDC-wei per ETH-wei
[yzABS] pb_raw=500.00000000000006 USDC per ETH -> 0 USDC-wei per ETH-wei
[yzABS] a=9.262096826685895 -> 9.262096826685894e-06 * scale = 10183783
[yzABS] b=22.360679774997898 -> 2.2360679774997898e-05 * scale = 24585827
[yzABS] yzABS = r(y=790000000, z=1100000000, A=10183783, B=24585827, S=1099511627776)


## Sundry tests of other carbonui stuff

In [6]:
oui = CarbonOrderUI.from_prices(P("ETH/USDC").sd(18,6), "USDC", 1000, 500, 1100, 790)
r = oui.set_id(1)
assert oui.tkn == "USDC"
assert oui.tkny == oui.tkn
assert oui.tknx == "ETH"
assert oui.pair.slashpair == "ETH/USDC"
assert abs(oui.S/9.262096826685895-1) < 1e-10
assert abs(oui.B/22.360679774997898-1) < 1e-10
assert oui.S is oui.A
assert abs(oui.pa/1000-1) < 1e-10
assert abs(oui.pa_raw/1000-1) < 1e-10
assert oui.p_start == oui.pa
assert oui.py == oui.pa
assert abs(oui.pb/500-1) < 1e-10
assert abs(oui.pb_raw/500-1) < 1e-10
assert oui.p_end == oui.pb
assert oui.py == oui.py
assert abs(oui.yint/1100-1) < 1e-10
assert abs(oui.y/790-1) < 1e-10
assert oui.y == oui.z
assert oui.total_liquidity == (790, 'USDC')
assert oui.price_convention == 'USDC per ETH'
assert oui.price_convention == oui.pair.price_convention
assert oui.price_convention == oui.price_convention_raw
assert tuple(oui.yzABS(32)) == (790000000, 1100000000, 39780, 96038, 4294967296)
assert r is oui
try:
    oui.set_id(1)
    raise
except ValueError as e:
    print(e)

('Curve ID has already been set', 1, 1)


In [7]:
oui.price_convention_raw

'USDC per ETH'

In [8]:
oui2 = CarbonOrderUI.from_prices(P("ETH/USDC").sd(18,6), "ETH", 1500, 2000, 2, 1)
oui2.set_id(2)
r  = oui.set_linked(oui2)
r2 = oui2.set_linked(oui)  # sets linked here and creates backlink
assert oui2.tkn == "ETH"
assert oui2.tkny == oui2.tkn
assert oui2.tknx == "USDC"
assert oui2.pair.slashpair == "ETH/USDC"
assert abs(oui2.S/0.0034592091997182155-1) < 1e-10
assert abs(oui2.B/0.022360679774997897-1) < 1e-10
assert oui2.S is oui2.A
assert abs(oui2.pa/1500-1) < 1e-10
assert abs(oui2.pa_raw/0.0006666666666666666-1) < 1e-10
assert oui2.p_start == oui2.pa
assert oui2.py == oui2.pa
assert abs(oui2.pb/2000-1) < 1e-10
assert abs(oui2.pb_raw/0.0005-1) < 1e-10
assert oui2.p_end == oui2.pb
assert oui2.py == oui2.py
assert abs(oui2.yint/2-1) < 1e-10
assert abs(oui2.y/1-1) < 1e-10
assert oui2.y == oui2.z
assert oui2.total_liquidity == (1, 'ETH')
assert oui2.price_convention == 'USDC per ETH'
assert oui2.price_convention == oui2.pair.price_convention
assert oui2.price_convention != oui2.price_convention_raw
assert oui2.price_convention_raw == "ETH per USDC"
assert tuple(oui2.yzABS(32)) == (1000000000000000000, 2000000000000000000, 14857190382812, 96038388349944, 4294967296)
assert r is oui
assert r2 is oui2
try:
    oui2.set_linked(oui)
    raise
except ValueError as e:
    print(e)

('Linked object has already been set', 1, 1)


In [9]:
assert oui.lid == 2
assert oui2.lid == 1

## Demo [NOTEST]

### Trade functions

In [10]:
mulDivF = lambda x, y, z: (x * y) // z
mulDiv = mulDivF
mulDivC = lambda x, y, z: (x * y + z - 1) // z

#### trade_by_source_dy_from_dx

In [11]:
def trade_by_source_dy_from_dx(params, C):

    dx = params[0]
    y,z,A,B,s = params[1]
    print(params)
    ONE = s
    temp1 = C(y * A + z * B, "temp1")               # 177 bits at most; cannot overflow
    temp2 = C(temp1 * dx / ONE, "temp2")            # 224 bits at most; can overflow; some precision loss
    temp3 = C(temp2 * A + z * z * ONE, "temp3")     # 256 bits at most; can overflow
    dy = mulDiv(temp1, temp2, temp3)
    print(dx, temp1, temp2, temp3, dy)
    return dy

In [12]:
# temp1 = z * ONE
# temp2 = y * A + z * B
# temp3 = temp2 * dy
# scale = mulDivC(temp3, A, 2**256-1)
# temp4 = mulDivC(temp1, temp1, scale)
# temp5 = mulDivC(temp3, A, scale)
# dx    = mulDivF(temp2, temp3 // scale, temp4 + temp5)

#### trade_by_target_dx_from_dy

this one is SKL version of the fixed code

In [13]:
def trade_by_target_dx_from_dy(params, C):

    dy = params[0]
    y,z,A,B,s = params[1]
    print(params)
    ONE = s
    temp1 = C(z * ONE, "temp1")                  
    temp2 = C(y * A + z * B, "temp2")         
    temp3 = C(temp2 - dy * A, "temp3")        
    scale = mulDiv(temp2, temp3, 2**255)+1
    temp1s = C(temp1//scale, "temp1s")
    temp2s = C(temp2//scale, "temp2s")
    dx = mulDiv(
        C(dy*temp1s, "dx*temp1s"), 
        temp1, 
        C(temp2s*temp3, "temp2s*temp3")
    )
    print(dy, dx*temp1s, temp1, temp2s*temp3, dx)
    return dx

this is BM version of the fixed code

In [14]:
# # trade_by_target_dx_from_dy
# mulDivF = lambda x, y, z: x * y // z
# mulDivC = lambda x, y, z: (x * y + z - 1) // z

# temp1 = z * ONE
# temp2 = y * A + z * B
# temp3 = temp2 - dx * A
# scale = mulDivC(temp2, temp3, 2**256-1)
# temp4 = mulDivC(temp1, temp1, scale)
# temp5 = mulDivF(temp2, temp3, scale)
# dy    = mulDivC(dx, temp4, temp5)

that's the previous, failing, version of the code

In [15]:
def trade_by_target_dx_from_dy_old(params, C):

    dy = params[0]
    y,z,A,B,s = params[1]
    ONE = s
    temp1 = C(z * ONE, "temp1")                  
    temp2 = C(y * A + z * B, "temp2")         
    temp3 = C(temp2 - dy * A, "temp3")        
    dx = mulDiv(
        C(dy*temp1, "dy*temp1"), 
        temp1, 
        C(temp2*temp3, "temp2*temp3")
    ) 
    print(dy, dy*temp1, temp1, temp2*temp3, dx)
    return dx

### Analysis

In [16]:
class STB(SolTestBase):
    #PRINT_LVL_DEFAULT = SolTestBase.LVL_LOG
    PRINT_LVL_DEFAULT = SolTestBase.LVL_WARN
VERBOSE = False

#### TKN/DAI -- same decimality, same reasonable price

In [17]:
TKNDAI = P("TKN/DAI").sd(18,18)
price = 5  # DAI per TKN
oui = CarbonOrderUI.from_prices(TKNDAI, "TKN", price, price, 1000, 1000)
c = curve = oui.yzABS(sx=40, verbose=VERBOSE)

In [18]:
params_bysrc  = (1e18, curve) # dy = wei sent (by source, 1 TKN)
dx = trade_by_target_dx_from_dy( params_bysrc, STB(context=("by_source", curve)))
dx/1e18 # 1 TKN -> 5 DAI ==> 5 DAI per TKN

(1e+18, r(y=1000000000000000000000, z=1000000000000000000000, A=0, B=491716548351, S=1099511627776))
1e+18 5497558138896018159282952339456000000000000000000000 1099511627776000000000000000000000 241785163922221334256682292088662337232306176000000000000000000000 5000000000014568431


5.000000000014569

In [19]:
params_bytarg = (5e18, curve)       # dx = token wei received (target, DAI)
dy = trade_by_source_dy_from_dx( params_bytarg, STB(context=("by_target", curve)) )
dy/1e18 # 1 TKN ->  5 DAI --> 5 DAI per TKN

(5e+18, r(y=1000000000000000000000, z=1000000000000000000000, A=0, B=491716548351, S=1099511627776))
5e+18 491716548351000000000000000000000 2236067977496531949120054474223665545216 1099511627776000000000000000000000000000000000000000000 999999999997086190


0.9999999999970862

#### ETH/USDC -- y=ETH, x=USDC

In [20]:
ETHUSDC = P("ETH/USDC").sd(18,6)
oui = CarbonOrderUI.from_prices(ETHUSDC, "ETH", 2000, 2000, 1, 0.5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)

In [21]:
params_bysrc  = (1*1e18, curve) # dy = token wei sent (by source, 1 ETH)
dx = trade_by_target_dx_from_dy( params_bysrc, STB(context=("by_source", curve)))
dx/1e6 # 1 ETH -> 2000 USDC

(1e+18, r(y=5e+17, z=1000000000000000000, A=0, B=24585827417585820, S=1099511627776))
1e+18 2199023254452488372224000000000000000000 1099511627776000000000000000000 604462909807314618688415159765465413263269547110658276245826332262400 1999999999


1999.999999

In [22]:
params_bytarg = (2000*1e6, curve)       # dx = token wei received (target, USDC)
dy = trade_by_source_dy_from_dx( params_bytarg, STB(context=("by_target", curve)) )
dy/1e18 # 1 ETH -> 2000 USDC

(2000000000.0, r(y=5e+17, z=1000000000000000000, A=0, B=24585827417585820, S=1099511627776))
2000000000.0 24585827417585819755229268861255680 44721359549995793980834690629632 1099511627776000000000000000000000000000000000000 1000000000000000027


1.0

#### ETH/USDC -- y=USDC, x=ETH

In [23]:
oui = CarbonOrderUI.from_prices(ETHUSDC, "USDC", 1000, 1000, 750, 750)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
print(curve)

r(y=750000000, z=750000000, A=0, B=34769610, S=1099511627776)


In [24]:
params_bysrc  = (1000*1e6, curve)       # dy = USDC-wei
dx = trade_by_target_dx_from_dy( params_bysrc, STB(context=("by_source", curve)) )
dx/1e18 # 1000 USDC -> 1 ETH

(1000000000.0, r(y=750000000, z=750000000, A=0, B=34769610, S=1099511627776))
1000000000.0 824633748159494423332333289472000000000 824633720832000000000 680020750998056250000000000000000 1000000033138948521


1.0000000331389485

In [25]:
params_bytarg = (1e18, curve)  # dx = ETH-wei
dy = trade_by_source_dy_from_dx( params_bytarg, STB(context=("by_target", curve)) )
dy/1e6 # 1000 USDC -> 1 ETH

(1e+18, r(y=750000000, z=750000000, A=0, B=34769610, S=1099511627776))
1e+18 26077207500000000 23717082058283267653632 618475290624000000000000000000 999999966


999.999966

#### SHIB/USDC -- y=SHIB, x=USDC

In [26]:
SHIBUSDC = P("SHIB/USDC").sd(18,6)
price = 1e-5
oui = CarbonOrderUI.from_prices(SHIBUSDC, "SHIB", price, price, 2e5, 2e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
print(curve)

r(y=2e+23, z=2e+23, A=0, B=347696105761141489664, S=1099511627776)


In [27]:
params_bysrc  = (1*1e5*1e18, curve)      # dy = SHIB-wei (1 USD worth of SHIB)
dx = trade_by_target_dx_from_dy( params_bysrc, STB(context=("by_source", curve)) )
dx / 1e6 # 1e5 SHIB -> 1 USDC

(1e+23, r(y=2e+23, z=2e+23, A=0, B=347696105761141489664, S=1099511627776))
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+23 2632807291694421164755057000000 219902325555199981553255926290448384 57896044618229089136646689796713055452480359437934408123369084530614903767040 1000000


1.0

In [28]:
params_bytarg = (1*1e6, curve) # dx = USDC-wei
dy = trade_by_source_dy_from_dx( params_bytarg, STB(context=("by_target", curve)) )
dy/1e18 # 1e5 SHIB -> 1 USDC

(1000000.0, r(y=2e+23, z=2e+23, A=0, B=347696105761141489664, S=1099511627776))
1000000.0 69539221152228296189580545547731059544686592 63245553203367578540979698731480776704 43980465111039994123099750822391432276334879391719429767168 99999999999999985943809


99999.99999999999

#### Nick's example DAI/USDC -- Selling DAI

In [29]:
DAIUSDC = P("DAI/USDC").sd(18,6)
price = 1
oui = CarbonOrderUI.from_prices(DAIUSDC, "DAI", price, price, 1e5, 1e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
print(curve)

r(y=1e+23, z=1e+23, A=0, B=1099511627776000000, S=1099511627776)


In [30]:
params_bysrc  = (1*1e18, curve)        # dy = DAI-wei
dx = trade_by_target_dx_from_dy( params_bysrc, STB(context=("by_source", curve)) )
dx/1e6 # 1 DAI -> 1 USDC

(1e+18, r(y=1e+23, z=1e+23, A=0, B=1099511627776000000, S=1099511627776))
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+18 526560283637935027904014110996239763 109951162777599990776627963145224192 57895973354467169559526534691796259748754773305955415093504382476762774241280 999999


0.999999

In [31]:
params_bytarg = (1*1e6, curve)  # dx = USDC-wei
dy = trade_by_source_dy_from_dx( params_bytarg, STB(context=("by_target", curve)) )
dy / 1e18 # 1 DAI -> 1 USDC

(1000000.0, r(y=1e+23, z=1e+23, A=0, B=1099511627776000000, S=1099511627776))
1000000.0 109951162777599990580649754106133915631616 99999999999999996863366107917975552 10995116277759998530774937705597858069083719847929857441792 1000000000000000016


1.0

#### Nick's example DAI/USDC -- Selling USDC

In [32]:
SHIBUSDC = P("DAI/USDC").sd(18,6)
price = 1
oui = CarbonOrderUI.from_prices(SHIBUSDC, "USDC", price, price, 1e5, 1e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)

In [33]:
params_bysrc  = (1*1e6, curve)       # dy = USDC-wei
dx = trade_by_target_dx_from_dy( params_bysrc, STB(context=("by_source", curve)) )
dx/1e18

(1000000.0, r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776))
1000000.0 109951288332907530323650989260800000000000 109951162777600000000000 12089244391210000000000000000000000 1000001141918869783


1.0000011419188697

In [34]:
params_bytarg = (1e18, curve)      # dx = DAI-wei
dy = trade_by_source_dy_from_dx( params_bytarg, STB(context=("by_target", curve)) )
dy/1e6

(1e+18, r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776))
1e+18 109951100000000000 99999942904105404465152 10995116277760000000000000000000000 999998


0.999998

## More examples [NOTEST]

In [35]:
class STB(SolTestBase):
    PRINT_LVL_DEFAULT = SolTestBase.LVL_LOG
    #PRINT_LVL_DEFAULT = SolTestBase.LVL_WARN
VERBOSE = True

### DAI/USDC

In [36]:
PAIR = P("DAI/USDC").sd(18,6)
price = 1
oui = CarbonOrderUI.from_prices(PAIR, "USDC", price, price, 1e5, 1e5)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
curve

[yzABS] pair=P('DAI/USDC').sd(18,6), y=USDC(6), x=DAI(18)
[yzABS] scale = 2**40 = 1099511627776
[yzABS] y=100000.0 -> y_wei=100000000000.0 [USDC-wei]
[yzABS] yint=100000.0 -> z_wei=100000000000.0 [USDC-wei]
[yzABS] pa_raw=1.0 USDC per DAI -> 0 USDC-wei per DAI-wei
[yzABS] pb_raw=1.0 USDC per DAI -> 0 USDC-wei per DAI-wei
[yzABS] a=0.0 -> 0.0 * scale = 0
[yzABS] b=1.0 -> 1e-06 * scale = 1099511
[yzABS] yzABS = r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776)


r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776)

In [37]:
params  = (1*1e6, curve)       # dy = USDC-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e18

(1000000.0, r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776))
[LOG:OK:TEMP1] 77 bits: ok
[LOG:OK:TEMP2] 57 bits: ok
[LOG:OK:TEMP3] 57 bits: ok
[LOG:OK:TEMP1S] 77 bits: ok
[LOG:OK:TEMP2S] 57 bits: ok
[LOG:OK:DX*TEMP1S] 97 bits: ok
[LOG:OK:TEMP2S*TEMP3] 114 bits: ok
1000000.0 109951288332907530323650989260800000000000 109951162777600000000000 12089244391210000000000000000000000 1000001141918869783


1.0000011419188697

In [38]:
params = (1e18, curve)      # dx = DAI-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e6

(1e+18, r(y=100000000000.0, z=100000000000.0, A=0, B=1099511, S=1099511627776))
[LOG:OK:TEMP1] 57 bits: ok
[LOG:OK:TEMP2] 77 bits: ok
[LOG:OK:TEMP3] 114 bits: ok
1e+18 109951100000000000 99999942904105404465152 10995116277760000000000000000000000 999998


0.999998

### SHIB/USDC

In [39]:
PAIR = P("SHIB/USDC").sd(18,6)
price = 1e-5 # SHIB per USDC
oui = CarbonOrderUI.from_prices(PAIR, "SHIB", price, price, 1e10, 1e10)
curve = oui.yzABS(sx=40, verbose=VERBOSE)
curve

[yzABS] pair=P('SHIB/USDC').sd(18,6), y=SHIB(18), x=USDC(6)
[yzABS] scale = 2**40 = 1099511627776
[yzABS] y=10000000000.0 -> y_wei=1e+28 [SHIB-wei]
[yzABS] yint=10000000000.0 -> z_wei=1e+28 [SHIB-wei]
[yzABS] pa_raw=99999.99999999999 SHIB per USDC -> 99999999999999984 SHIB-wei per USDC-wei
[yzABS] pb_raw=99999.99999999999 SHIB per USDC -> 99999999999999984 SHIB-wei per USDC-wei
[yzABS] a=0.0 -> 0.0 * scale = 0
[yzABS] b=316.2277660168379 -> 316227766.0168379 * scale = 347696105761141489664
[yzABS] yzABS = r(y=1e+28, z=1e+28, A=0, B=347696105761141489664, S=1099511627776)


r(y=1e+28, z=1e+28, A=0, B=347696105761141489664, S=1099511627776)

In [40]:
params  = (1*1e5*1e18, curve)       # dy = SHIB-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e6

(1e+23, r(y=1e+28, z=1e+28, A=0, B=347696105761141489664, S=1099511627776))
[LOG:OK:TEMP1] 134 bits: ok
[LOG:OK:TEMP2] 162 bits: ok
[LOG:OK:TEMP3] 162 bits: ok
[LOG:OK:TEMP1S] 66 bits: ok
[LOG:OK:TEMP2S] 94 bits: ok
[LOG:OK:DX*TEMP1S] 142 bits: ok
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+23 52656145834278616434000000 10995116277759999541635303256465061445632 57896044618658097711558570309110167386784032288136095717007925787226802225152 1000000


1.0

In [41]:
params = (1e6, curve)      # dx = USDC-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e18

(1000000.0, r(y=1e+28, z=1e+28, A=0, B=347696105761141489664, S=1099511627776))
[LOG:OK:TEMP1] 162 bits: ok
[LOG:OK:TEMP2] 142 bits: ok
[LOG:OK:TEMP3] 227 bits: ok
1000000.0 3476961057611414482900541393588953416646166315008 3162277660168378678463613278315914786242560 109951162777599986159580753732140146302259305055688004140916581335040 99999999999999967915471


99999.99999999997

### SHIB/BTC (selling SHIB)

In [42]:
PAIR = P("SHIB/BTC").sd(18,8)
price = 1e-5 * 1e-5 # SHIB per BTC # 1e10 USD
capacity = 1000 * 1e10 # 1000 BTC
oui = CarbonOrderUI.from_prices(PAIR, "SHIB", price, price*1.05, capacity, capacity)
curve = oui.yzABS(sx=40, verbose=True)
curve

[yzABS] pair=P('SHIB/BTC').sd(18,8), y=SHIB(18), x=BTC(8)
[yzABS] scale = 2**40 = 1099511627776
[yzABS] y=10000000000000.0 -> y_wei=1e+31 [SHIB-wei]
[yzABS] yint=10000000000000.0 -> z_wei=1e+31 [SHIB-wei]
[yzABS] pa_raw=9999999999.999996 SHIB per BTC -> 99999999999999967232 SHIB-wei per BTC-wei
[yzABS] pb_raw=9523809523.80952 SHIB per BTC -> 95238095238095208448 SHIB-wei per BTC-wei
[yzABS] a=2409.9927051466802 -> 240999270.51466802 * scale = 264981500216411193344
[yzABS] b=97590.0072948533 -> 9759000729.48533 * scale = 10730134777543587266560
[yzABS] yzABS = r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776)


r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776)

#### Trading 1e10 SHIB -> 1 BTC

In [43]:
params  = (1*1e5*1e5*1e18, curve)       # dy = SHIB-wei (1USD)
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e8 # 1e10 SHIB = 1 BTC

(1e+28, r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776))
[LOG:OK:TEMP1] 143 bits: ok
[LOG:OK:TEMP2] 177 bits: ok
[LOG:OK:TEMP3] 177 bits: ok
[LOG:OK:TEMP1S] 46 bits: ok
[LOG:OK:TEMP2S] 79 bits: ok
[LOG:OK:DX*TEMP1S] 139 bits: ok
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+28 5265868391783346665350 10995116277759999599663742597967261831528448 57896044618658097711785488090213151806879688637054915063744949408839097122816 100002410


1.0000241

In [44]:
params = (1e8, curve)      # dx = BTC-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e18/1e10 #1e10 SHIB = 1 BTC

(100000000.0, r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776))
[LOG:OK:TEMP1] 177 bits: ok
[LOG:OK:TEMP2] 163 bits: ok
[WARNING:OVERFLOW:TEMP3] 246 bits: close to max
100000000.0 109951162777599979414058781985324393505188997761073152 9999999999999998166828554841774886064248332484608 109953812592602152227049417074515479289564729561435005246035576444919218176 9999759006537407575786154669


0.9999759006537406

#### Trading 1e5 SHIB -> 1e-5 BTC (1USD)

In [45]:
params  = (1*1e5*1e18, curve)       # dy = SHIB-wei (1USD)
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e8/1e-5 # 1e5 SHIB = 1e-5 BTC

(1e+23, r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776))
[LOG:OK:TEMP1] 143 bits: ok
[LOG:OK:TEMP2] 177 bits: ok
[LOG:OK:TEMP3] 177 bits: ok
[LOG:OK:TEMP1S] 46 bits: ok
[LOG:OK:TEMP2S] 79 bits: ok
[LOG:OK:DX*TEMP1S] 122 bits: ok
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+23 52656145846968000 10995116277759999599663742597967261831528448 57896044618658097711785437676134072002576463420442807956216976457418753966080 1000


1.0

In [46]:
params = (1e-5*1e8, curve)      # dx = BTC-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e18/1e5 # 1e5 SHIB = 1e-5 BTC

(1000.0000000000001, r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776))
[LOG:OK:TEMP1] 177 bits: ok
[LOG:OK:TEMP2] 147 bits: ok
[WARNING:OVERFLOW:TEMP3] 246 bits: close to max
1000.0000000000001 109951162777599979414058781985324393505188997761073152 99999999999999989014320776740338242315878400 109951162804098136008063670117694473318988536657126038699479705259144642560 99999999975900055991132


0.9999999997590007

#### Trading 1e3 SHIB -> 1e-7 BTC (0.01USD)

In [47]:
params  = (1*1e3*1e18, curve)       # dy = SHIB-wei (1USD cent)
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e8/1e-7 # 1e3 SHIB = 1e-7 BTC

(1e+21, r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776))
[LOG:OK:TEMP1] 143 bits: ok
[LOG:OK:TEMP2] 177 bits: ok
[LOG:OK:TEMP3] 177 bits: ok
[LOG:OK:TEMP1S] 46 bits: ok
[LOG:OK:TEMP2S] 79 bits: ok
[LOG:OK:DX*TEMP1S] 116 bits: ok
[WARNING:OVERFLOW:TEMP2S*TEMP3] 255 bits: close to max
1e+21 526561458344050 10995116277759999599663742597967261831528448 57896044618658097711785490451715848511135079116478974928505173379055284649984 10


1.0

In [48]:
params = (1e-7*1e8, curve)      # dx = BTC-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e18/1e3 # 1e3 SHIB = 1e-7 BTC

(10.0, r(y=1e+31, z=1e+31, A=264981500216411193344, B=10730134777543587266560, S=1099511627776))
[LOG:OK:TEMP1] 177 bits: ok
[LOG:OK:TEMP2] 140 bits: ok
[WARNING:OVERFLOW:TEMP3] 246 bits: close to max
10.0 109951162777599979414058781985324393505188997761073152 999999999999999890143207767403382423158784 109951162777864972869163420056364163534149828493613732463662723647401885696 999999999997589788708


0.9999999999975898

### SHIB/BTC (selling BTC)

In [49]:
PAIR = P("SHIB/BTC").sd(18,8)
price = 1e-5 * 1e-5 # SHIB per BTC
capacity = 1000 # 1000 BTC
oui = CarbonOrderUI.from_prices(PAIR, "BTC", price, price/1.05, capacity, capacity)
curve = oui.yzABS(sx=48, verbose=True)
curve

[yzABS] pair=P('SHIB/BTC').sd(18,8), y=BTC(8), x=SHIB(18)
[yzABS] scale = 2**48 = 281474976710656
[yzABS] y=1000 -> y_wei=100000000000 [BTC-wei]
[yzABS] yint=1000 -> z_wei=100000000000 [BTC-wei]
[yzABS] pa_raw=1.0000000000000002e-10 BTC per SHIB -> 0 BTC-wei per SHIB-wei
[yzABS] pb_raw=9.523809523809526e-11 BTC per SHIB -> 0 BTC-wei per SHIB-wei
[yzABS] a=2.4099927051466823e-07 -> 2.4099927051466825e-12 * scale = 678
[yzABS] b=9.759000729485333e-06 -> 9.759000729485333e-11 * scale = 27469
[yzABS] yzABS = r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656)


r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656)

#### Trading 1 BTC -> 1e10 SHIB

In [50]:
params  = (1*1e8, curve)       # dy = BTC-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e18/1e10 # 1e10 SHIB <- 1 BTC

(100000000.0, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 85 bits: ok
[LOG:OK:TEMP2] 52 bits: ok
[LOG:OK:TEMP3] 52 bits: ok
[LOG:OK:TEMP1S] 85 bits: ok
[LOG:OK:TEMP2S] 52 bits: ok
[LOG:OK:DX*TEMP1S] 112 bits: ok
[LOG:OK:TEMP2S*TEMP3] 103 bits: ok
100000000.0 281491710918920126547675841886316090779238400000000000 28147497671065600000000000 7922345253340000000000000000000 10000594518506038587719183464


1.0000594518506039

In [51]:
params = (1e10*1e18, curve)      # dx = SHIB-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e8 # 1e3 SHIB <- 1 BTC

(1e+28, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 52 bits: ok
[LOG:OK:TEMP2] 97 bits: ok
[LOG:OK:TEMP3] 122 bits: ok
1e+28 2814700000000000 99998231917197704277356707840 2814817565907799860043500047847915520 99994055


0.99994055

#### Trading 1e-5 BTC -> 1e5 SHIB (1USD)

In [52]:
params  = (1e-5*1e8, curve)       # dy = BTC-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e18/1e5 # 1e5 SHIB <- 1e-5 BTC

(1000.0000000000001, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 85 bits: ok
[LOG:OK:TEMP2] 52 bits: ok
[LOG:OK:TEMP3] 52 bits: ok
[LOG:OK:TEMP1S] 85 bits: ok
[LOG:OK:TEMP2S] 52 bits: ok
[LOG:OK:DX*TEMP1S] 95 bits: ok
[LOG:OK:TEMP2S*TEMP3] 103 bits: ok
1000.0000000000001 2814849304637549593419477657807591833600000000000 28147497671065600000000000 7922536088091633400000000000000 100003536283479007573031


1.0000353628347902

In [53]:
params = (1e5*1e18, curve)      # dx = SHIB-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e8/1e-5 # 1e3 SHIB <- 1e-5 BTC

(1e+23, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 52 bits: ok
[LOG:OK:TEMP2] 80 bits: ok
[LOG:OK:TEMP3] 122 bits: ok
1e+23 2814700000000000 999982319171976944418816 2814749767784548012398600368315957248 999


0.9989999999999999

#### Trading 1e-7 BTC -> 1e3 SHIB (0.01USD)

In [54]:
params  = (1e-7*1e8, curve)       # dy = BTC-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e18/1e3 # 1e3 SHIB <- 1e-7 BTC

(10.0, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 85 bits: ok
[LOG:OK:TEMP2] 52 bits: ok
[LOG:OK:TEMP3] 52 bits: ok
[LOG:OK:TEMP1S] 85 bits: ok
[LOG:OK:TEMP2S] 52 bits: ok
[LOG:OK:DX*TEMP1S] 88 bits: ok
[LOG:OK:TEMP2S*TEMP3] 103 bits: ok
10.0 28148493039662935489981689861085593600000000000 28147497671065600000000000 7922536089980916334000000000000 1000035362596312022631


1.000035362596312

In [55]:
params = (1e3*1e18, curve)      # dx = SHIB-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e8/1e-7 # 1e3 SHIB <- 1e-7 BTC

(1e+21, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 52 bits: ok
[LOG:OK:TEMP2] 74 bits: ok
[LOG:OK:TEMP3] 122 bits: ok
1e+21 2814700000000000 9999823191719769276416 2814749767113339880123986003569410048 9


0.9

#### Trading 10 BTC -> 1e11 SHIB

In [56]:
params  = (10*1e8, curve)       # dy = BTC-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e18/1e11 # 1e11 SHIB <- 10 BTC

(1000000000.0, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 85 bits: ok
[LOG:OK:TEMP2] 52 bits: ok
[LOG:OK:TEMP3] 52 bits: ok
[LOG:OK:TEMP1S] 85 bits: ok
[LOG:OK:TEMP2S] 52 bits: ok
[LOG:OK:DX*TEMP1S] 115 bits: ok
[LOG:OK:TEMP2S*TEMP3] 103 bits: ok
1000000000.0 2815527503287053823350219933470347590683852800000000000 28147497671065600000000000 7920627723400000000000000000000 100027630739669384615461209388


1.0002763073966938

In [57]:
params = (1e11*1e18, curve)      # dx = SHIB-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e8/10 # 1e3 SHIB <- 100 BTC

(1e+29, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 52 bits: ok
[LOG:OK:TEMP2] 100 bits: ok
[LOG:OK:TEMP3] 122 bits: ok
1e+29 2814700000000000 999982319171976972404822900736 2815427755118958600387290469926699008 999723835


0.999723835

#### Trading 100 BTC -> 1e12 SHIB

In [58]:
params  = (100*1e8, curve)       # dy = BTC-wei
dx = trade_by_target_dx_from_dy( params, STB(context=("by_source", curve)) )
dx/1e18/1e12 # 1e12 SHIB <- 100 BTC

(10000000000.0, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 85 bits: ok
[LOG:OK:TEMP2] 52 bits: ok
[LOG:OK:TEMP3] 52 bits: ok
[LOG:OK:TEMP1S] 85 bits: ok
[LOG:OK:TEMP2S] 52 bits: ok
[LOG:OK:DX*TEMP1S] 118 bits: ok
[LOG:OK:TEMP2S*TEMP3] 103 bits: ok
10000000000.0 28216460354478908851005842750710385782607052800000000000 28147497671065600000000000 7903452424000000000000000000000 1002450046686892507743701328263


1.0024500466868926

In [59]:
params = (1e12*1e18, curve)      # dx = SHIB-wei
dy = trade_by_source_dy_from_dx( params, STB(context=("by_target", curve)) )
dy/1e8/100 # 1e3 SHIB <- 100 BTC

(1e+30, r(y=100000000000, z=100000000000, A=678, B=27469, S=281474976710656))
[LOG:OK:TEMP1] 52 bits: ok
[LOG:OK:TEMP2] 103 bits: ok
[LOG:OK:TEMP3] 122 bits: ok
1e+30 2814700000000000 9999823191719770286998182428672 2821529647230546004254584767686639616 9975618142


0.9975618142

## SolTestBase tests

In [60]:
TB0 = SolTestBase()
TBrw = SolTestBase(raise_lvl=TB0.LVL_WARN)
TBre = SolTestBase(raise_lvl=TB0.LVL_ERR)

In [61]:
try:
    TBrw.check_uint256(1, "testlabel")
    raise RuntimeError("should raise")
except TB0.UnderflowError as e:
    print(e)
try:
    TBrw.check_uint256(2000, "testlabel")
    raise RuntimeError("should raise")
except TB0.UnderflowWarning as e:
    print(e)
try:
    TBrw.check_uint256(2**254, "testlabel")
    raise RuntimeError("should raise")
except TB0.OverflowWarning as e:
    print(e)
try:
    TBrw.check_uint256(2**256-1, "testlabel")
    raise RuntimeError("should raise")
except TB0.OverflowWarning as e:
    print(e)
try:
    TBrw.check_uint256(2**256, "testlabel")
    raise RuntimeError("should raise")
except TB0.OverflowError as e:
    print(e)

[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold
('[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold', {}, None)
[WARNING:UNDERFLOW:TESTLABEL] 11 bits: underflow warning
('[WARNING:UNDERFLOW:TESTLABEL] 11 bits: underflow warning', {}, None)
[WARNING:OVERFLOW:TESTLABEL] 254 bits: close to max
('[WARNING:OVERFLOW:TESTLABEL] 254 bits: close to max', {}, None)
[WARNING:OVERFLOW:TESTLABEL] 256 bits: close to max
('[WARNING:OVERFLOW:TESTLABEL] 256 bits: close to max', {}, None)
[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max
('[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max', {}, None)


In [62]:
try:
    TBre.check_uint256(1, label="testlabel")
    raise RuntimeError("should raise")
except TBre.UnderflowError as e:
    print(e)
assert TBre.check_uint256(2000, label="testlabel") == 2000
assert TBre.check_uint256(2**254, label="testlabel") == 2**254
assert TBre.check_uint256(2**256-1, label="testlabel") == 2**256-1
try:
    TBre.check_uint256(2**256, label="testlabel")
    raise RuntimeError("should raise")
except TBre.OverflowError as e:
    print(e)

[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold
('[ERROR:UNDERFLOW:TESTLABEL] 1 bits: < underflow threshold', {}, None)
[WARNING:UNDERFLOW:TESTLABEL] 11 bits: underflow warning
[WARNING:OVERFLOW:TESTLABEL] 254 bits: close to max
[WARNING:OVERFLOW:TESTLABEL] 256 bits: close to max
[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max
('[ERROR:OVERFLOW:TESTLABEL] 256 bits: > max', {}, None)


In [63]:
assert TB0.check_uint256(1, log_f=TB0.print_f) == 1
assert TB0.check_uint256(2000, log_f=TB0.print_f) == 2000
assert TB0.check_uint256(2**254, log_f=TB0.print_f) == 2**254
assert TB0.check_uint256(2**256-1, log_f=TB0.print_f) == 2**256-1
assert TB0.check_uint256(2**256, log_f=TB0.print_f) == 2**256

[ERROR:UNDERFLOW:] 1 bits: < underflow threshold
[WARNING:UNDERFLOW:] 11 bits: underflow warning
[WARNING:OVERFLOW:] 254 bits: close to max
[WARNING:OVERFLOW:] 256 bits: close to max
[ERROR:OVERFLOW:] 256 bits: > max


In [64]:
assert TB0._logmsg(level=TB0.LVL_WARN, isoverflow=True, label="1", msg="") == '[WARNING:OVERFLOW:1] '
assert TB0._logmsg(TB0.LVL_ERR, False, "mylabel", "mymessage") == '[ERROR:UNDERFLOW:MYLABEL] mymessage'
TB0.print_f(False, False, "mylabel", "mymessage")

In [65]:
assert TB0.bindig(0) == 0
assert TB0.bindig(1) == 1
assert TB0.bindig(2) == 2
assert TB0.bindig(3) == 2
assert TB0.bindig(2**10) == 11
assert TB0.bindig(2**10+1) == 11
assert TB0.bindig(2**10-1) == 10